# Spelling correction

The following files are required:
- sentences_with_typos.txt: a file with two tab-separated columns, the first containing a numerical ID and the second containing a sentence in English
- SUBTLEXus.txt: a file with several tab-separated columns, containing data from the SUBTLEXus lexicon, a list of words derived from a part of the SUBTLEXus corpus containing movie subtitles with their frequency counts (and other pieces of information)
- LM_trainingCorpus.json: a file containing a pre-processed corpus in the form of a list of lists of strings, to be used to train a word-level statistical language model


## Task1

In the cell below you find code to run a statistical language model, add the docstrings (you already find a blueprint) to complete the first task.

In [1]:
import operator
import numpy as np
from collections import defaultdict

class LM(object):
    
    def __init__(self, corpus, ngram_size=2, bos='+', eos='#', k=1):
        
        """        
        :param corpus: List of sentences, where each sentence is a list of tokens.
        :param ngram_size: The size of the n-grams to be used for the language model (default is bigrams).
        :param bos: Beginning of sentence (default is '+').
        :param eos: End of sentence (default is '#').
        :param k: Smoothing constant for Laplace smoothing (default is 1).
        
        This class creates an LM object with attributes for setting the corpus, n-gram size, bos and eos tokens, 
        and Laplace smoothing constant k.
        """
        
        self.k = k
        self.ngram_size = ngram_size
        self.bos = bos
        self.eos = eos
        self.corpus = corpus
        self.vocab = self.get_vocab()
        self.vocab.add(self.eos)
        self.vocab_size = len(self.vocab)
        
    def get_vocab(self):
        
        """        
        :return: A set that contains the unique tokens in given corpus (vocab).
        
        This method derives the vocabulary set from the input corpus by iterating through each sentence and 
        each token (element) within the selected sentence, then add the constituant elements to vocab set.
        """
        
        vocab = set()
        for sentence in self.corpus:
            for element in sentence:
                vocab.add(element)
        
        return vocab
                    
    def update_counts(self):

        """
        :return: None.
        
        
        This method creates a 'counts' using defaultdict with the n-gram counts for the training corpus. The counts is
        a nested dictionary with keys are tuples containing the first n-gram -1 words and the value is a dictionary
        containing the next word(s) with their counts. 
        
        The method also add padding to each sentence in the corpus before processing. Pedding is important to keep
        track of the start and end of a sentence.
        
        This (counts) dictionary is useful for calculating n-gram probabilities and perplexities.
        """
        
        r = self.ngram_size - 1
        
        self.counts = defaultdict(dict)
    
        for sentence in self.corpus:
            s = [self.bos]*r + sentence + [self.eos]
            
            for idx in range(self.ngram_size-1, len(s)):
                ngram = self.get_ngram(s, idx)
                
                try:
                    self.counts[ngram[0]][ngram[1]] += 1
                except KeyError:
                    self.counts[ngram[0]][ngram[1]] = 1
                        
    def get_ngram(self, s, i):
        
        """        
        :param s: Sentence represented as a list of tokens include peddings.
        :param i: Index in the sentence where the n-gram ends.
        :return: A tuple containing the n-gram history and target word.
        
        This method retrieves the n-gram at a given index in a sentence, slice it and return its  
        history and the target as a tuple.
        """
        
        ngram = s[i-(self.ngram_size-1):i+1]
        history = tuple(ngram[:-1])
        target = ngram[-1]
        return (history, target)
    
    def get_ngram_probability(self, history, target):
        
        """        
        :param history: A tuple containing the history of the n-gram.
        :param target: The target token of the n-gram.
        :return: The probability of the target token given its history.
        
        This method calculates the probability of a target token given its n-gram history.
        It uses the n-gram counts and applies constant k (Laplace smoothing) to estimate the probability
        of a word occurring in a given context.
        
        """
        try:
            ngram_tot = np.sum(list(self.counts[history].values())) + (self.vocab_size*self.k)
            try:
                transition_count = self.counts[history][target] + self.k
            except KeyError:
                transition_count = self.k
        except KeyError:
            transition_count = self.k
            ngram_tot = self.vocab_size*self.k

        return transition_count/ngram_tot 

    def perplexity(self, sentence):
        
        """
        :param sentence: A list of tokens representing the sentence.
        :return: The perplexity of the input sentence.

        This method computes the preplexity measure as an evaluation of a the language model's ability to predict
        the given sentence. It does this by calculating the log probability of each n-gram using (get_ngram_probability)
        method and then adding the log to 'probs' (list). The perplexity obtained by taking the exponent of the
        negative average of the log probabilities 'avg_entropy'.
        """

        r = self.ngram_size - 1
        s = [self.bos]*r + sentence + [self.eos]

        probs = []
        for idx in range(self.ngram_size-1, len(s)):
            ngram = self.get_ngram(s, idx)
            probs.append(self.get_ngram_probability(ngram[0], ngram[1]))

        entropy = np.log2(probs)
        avg_entropy = -1 * (sum(entropy) / len(entropy))
        return pow(2.0, avg_entropy)

In [2]:
import json
import nltk
import spacy
import pandas as pd
from collections import defaultdict

## Task2

In [3]:
# load the data

sentences_with_typos_df = pd.read_csv("CL/sentence_with_typos.txt", sep="\t", keep_default_na=False)
SUBTLEXus_df = pd.read_csv("CL/SUBTLEXus.txt", sep="\t", keep_default_na=False, low_memory=False)

with open("CL/LM_trainingCorpus.json", "r") as f:
    LM_training_corpus = json.load(f)

In [4]:
print(sentences_with_typos_df.head())

   ID                                           sentence
0   1           did you knoq that our fiends had a baby?
1   2  I red that reserachers managed to deviate the ...
2   3  I could not help but grozn in frustration when...
3   4  I mate a quolg from old clothes for my newborn...
4   5  he waies his wand to get the attention of the ...


In [5]:
print(SUBTLEXus_df.head())

  Word  FREQcount  CDcount  FREQlow  Cdlow   SUBTLWF  Lg10WF  SUBTLCD  Lg10CD
0  the    1501908     8388  1339811   8388  29449.18  6.1766   100.00  3.9237
1   to    1156570     8383  1138435   8380  22677.84  6.0632    99.94  3.9235
2    a    1041179     8382   976941   8380  20415.27  6.0175    99.93  3.9234
3  you    2134713     8381  1595028   8376  41857.12  6.3293    99.92  3.9233
4  and     682780     8379   515365   8374  13387.84  5.8343    99.89  3.9232


In [6]:
print(LM_training_corpus[:5])

[['He', 'did', ',', 'however', ',', 'propel', 'himself', 'full', 'force', 'into', 'an', 'unoccupied', 'table', ',', 'bringing', 'it', 'crashing', 'to', 'the', 'ground', ',', 'its', 'umbrella', ',', 'a', 'chair', ',', 'cutlery', ',', 'salt', 'and', 'pepper', 'shakers', ',', 'packets', 'of', 'sugar', ',', 'and', 'other', 'odds', 'and', 'ends', 'falling', 'on', 'top', 'of', 'him', '.'], ['"', 'You', 'told', 'someone', '.'], ['Despite', 'the', 'gun', 'in', 'his', 'face', ',', 'Corbett', 'remains', 'calm', '.'], ['"', 'I', 'waited', 'as', 'he', 'savored', 'another', 'squirt', ',', 'knowing', 'he', 'had', "n't", 'p39', 'root', 'of', 'questions', '.'], ['"', 'But', 'those', 'seeds', '.']]


## Task 2a

After reading in the files as Pandas DataFrames (mind how you import default strings, some of them may turn into NAs!), find which token contains a typo in the given sentences, where having a typo means that the word does not feature in SUBTLEXus - remember to tokenise the input sentences! There is one and only one word containing a typo in each sentence, as defined in this way: the typo can result from the insertion/deletion/substitution of one or two characters. 

You should submit a .json file containing a simple dictionary mapping the sentence ID (as an integer) to the mistyped word (as a string). 5 points available in total: for every incorrect word retrieved, you lose 0.5 point. If you retrieve 10 wrong mistyped words, you get no points even if the total number of mistyped words is higher than 10. If you submit a wrongly formatted file, you will get no points.

In [15]:
# find mistyped words in the input sentences

import string

SUBTLEXus_words = set(SUBTLEXus_df["Word"])

mistyped_words = defaultdict(str)

def lexicon_lookup(tokens):
    for token in tokens:
        if token not in SUBTLEXus_words and token not in string.punctuation:
            return token


In [26]:
for i, row in sentences_with_typos_df.iterrows():
    sentence_id = row["ID"]
    sentence = row["sentence"]
    
    tokens = nltk.word_tokenize(sentence)
    #print("index: {}, row: {}".format(index, row), tokens)
    
    mistyped_words[sentence_id] = lexicon_lookup(tokens)

In [27]:
print(mistyped_words)

defaultdict(<class 'str'>, {1: 'knoq', 2: 'reserachers', 3: 'grozn', 4: 'quolg', 5: 'waies', 6: 'wintr', 7: 'munors', 8: 'surgicaly', 9: 'aquire', 10: 'acomodate', 11: 'dats', 12: 'collegue', 13: 'layed', 14: 'cate', 15: 'giambic'})


In [28]:
with open("task2a_GhaithAlSeirawan_solution.json", "w") as f:
    json.dump(mistyped_words, f)

## Task2b

read the target sentences manually. Some of them contain another mistyped word than the one you found in task2a, but were not detected because the typo resulted in a word which appears in SUBTLEXus. Discuss how you could automatically spot those mistyped words too using CL methods and resources: what information would you need? How would you use it? Reply in no more than 150 words. 3 points available, awarded based on how sensible the answer is.

### Answer
We can use a statistical language model (LM) trained on a big, corpus (such the provided LM_trainingCorpus.json) to automatically detect mistyped words even when they do not exist in SUBTLEXus. 
An LM can assist in locating words that are probably mistyped by making use of the context of the words around them. Thus, we can compute the perplexity or the probability of each word in the sentence using a trained language model on a large corpus. If a word has a high perplexity or lower probability than others in the same context we could indicate it as mistyped word. A threshold can be set to improve detection based on the distribution of perplexities or probabilities in the corpus.


## Task 3

find the words from SUBTLEXus with the smallest edit distance from each mistyped target (do not lowercase anything). You should return the 3 words at the smallest edit distance, sorted by edit distance. However, if there are more words at the same edit distance than the third closest, you should include all the words at the same edit distance.

    Therefore, supposing that the string 'abcdef' has two neighbors at edit distance 1, and four neighbors at edit distance 2, the third closest neighbor would be at edit distance 2, but there would be other three words at the same distance and you should thus return six neighbors for the target string. 
5 points available: you loose 0.5 points for every wrong list of nearest neighbors you retrieve for a mistyped word (wrong means any of wrong words, wrong order, wrong edit distances, wrong data type).
Submit a .json file containing a dictionary mapping sentence IDs (as integers) to lists of tuples, where each tuple contains first the word (as a string) and then the edit distance (as an integer), with tuples sorted by edit distance in ascending order (smallest edit distance first). The dictionary should have the following form (if you submit a wrongly formatted file, you will get no points): {id1: [(w1, 2), (w2, 2), (w3, 2)]; id2: [(w6, 1), (w7, 1), (w8, 2), (w9, 2), (w10, 2)]; ...}

In [44]:
# retrieve the nearest neighbors based on edit distance

closest_neighbors = defaultdict(list)

def find_edit_distance(mistyped_word):
    target_words = list()
    
    for target_word in SUBTLEXus_words:
        target_words.append((target_word, nltk.edit_distance(mistyped_word, target_word)))
        
    return target_words



In [ ]:
for sen_id, mistyped_word in mistyped_words.items():
    distances = find_edit_distance(mistyped_word)
    #print(distances[:20])
    distances.sort(key = lambda x: x[1])
    
    top_neighbors = [distances[0], distances[1], distances[2]]
    #print("top_neighbors ", top_neighbors)
    
    smallest_dist = distances[2][1]
    #print("smallest_dist ", smallest_dist)
    
    for dist in distances[3:]:
        if dist[1] == smallest_dist:
            top_neighbors.append(dist)

    closest_neighbors[int(sen_id)] = top_neighbors


In [45]:
print(closest_neighbors)

defaultdict(<class 'list'>, {1: [('know', 1), ('knot', 1), ('knob', 1)], 2: [('researchers', 2), ('researcher', 3), ('reachers', 3), ('researches', 3)], 3: [('grown', 1), ('groin', 1), ('groan', 1)], 4: [('quoit', 2), ('quote', 2), ('quota', 2), ('qualm', 2), ('quoad', 2), ('quos', 2), ('quila', 2), ('quilt', 2), ('quo', 2), ('quell', 2)], 5: [('waifs', 1), ('wails', 1), ('wakes', 1), ('waits', 1), ('wares', 1), ('waives', 1), ('wades', 1), ('wanes', 1), ('waves', 1), ('waxes', 1), ('wages', 1)], 6: [('wintry', 1), ('winter', 1), ('wants', 2), ('width', 2), ('dinar', 2), ('wanty', 2), ('dint', 2), ('winds', 2), ('contr', 2), ('tints', 2), ('entr', 2), ('wite', 2), ('diner', 2), ('wing', 2), ('wilts', 2), ('intro', 2), ('tint', 2), ('wiper', 2), ('minty', 2), ('wist', 2), ('linty', 2), ('Pinto', 2), ('bint', 2), ('wiser', 2), ('wined', 2), ('wilt', 2), ('wink', 2), ('winder', 2), ('aint', 2), ('with', 2), ('Sinter', 2), ('hints', 2), ('wit', 2), ('wintery', 2), ('wiener', 2), ('liner', 

In [46]:
with open("task3_GhaithAlSeirawan_solution.json", "w") as f:
    json.dump(closest_neighbors, f)

## Task4: frequency

Use the list of candidate replacements you found in task3 to find the best one according to candidate frequency (derived from SUBTLEXus)

- If two or more candidates have the exact same frequency in SUBTLEX, choose the one with the best edit distance. 
- If two or more candidates at the same frequency also have the same edit distance, pick the one which comes first in alphabetical order.

You should return a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its SUBTLEXus frequency value (as an integer). 5 points available, you lose 1 point for each wrong best candidate:frequency pair you retrieve (if the candidate is right but the frequency doesn't match, you loose half a point).

In [47]:
# this is just a helping function for debugging
from itertools import islice

def take(n, iterable):
    return list(islice(iterable, n))

In [48]:
# pick the best candidate according to SUBTLEXus frequency counts

# I will be using "Lg10WF" for this task instead of computing the frequency counts manually.
# The "Lg10WF" is log10(FREQcount+1) according to https://www.ugent.be/pp/experimentele-psychologie/en/research/documents/subtlexus


SUBTLEXus_freq = SUBTLEXus_df.set_index("Word")["Lg10WF"].to_dict()

# print(take(5, SUBTLEXus_freq))
# print(SUBTLEXus_freq['the'])
# print(SUBTLEXus_freq.get('the'))


def find_best_candidate_WF(neighbors, lexicon_WF):
    best_candidate = None
    best_frequency = -1
    best_edit_distance = float("inf")

    for candidate, edit_distance in neighbors:
        frequency = lexicon_WF.get(candidate)

        if (
            (frequency > best_frequency)
            or (frequency == best_frequency and edit_distance < best_edit_distance) 
            or (
                frequency == best_frequency
                and edit_distance == best_edit_distance
                and candidate < best_candidate
               )
           ):
            
            best_candidate = candidate
            best_frequency = frequency
            best_edit_distance = edit_distance
            
    return (best_candidate, best_frequency)


In [50]:
best_candidate_replacements = {}

for sen_id, neighbors in closest_neighbors.items():
    best_candidate_replacements[sen_id] = find_best_candidate_WF(neighbors, SUBTLEXus_freq)


In [51]:
print(best_candidate_replacements)

{1: ('know', 5.4651), 2: ('researchers', 1.7634), 3: ('grown', 3.1059), 4: ('quote', 2.6893), 5: ('waves', 2.8293), 6: ('with', 5.4107), 7: ('minor', 2.8162), 8: ('strictly', 2.7396), 9: ('Squire', 2.1987), 10: ('accommodate', 2.0414), 11: ('days', 4.1929), 12: ('college', 3.638), 13: ('played', 3.458), 14: ('care', 4.3936), 15: ('gambit', 1.301)}


In [52]:

with open("task4_GhaithAlSeirawan_solution.json", "w") as f:
    json.dump(best_candidate_replacements, f)

## Task5: perplexity

Use the list of candidate replacements you found in task3 to find the best one according to its perplexity under a statistical language model of order 3 implemented using a Markov Chain with add-k smoothing (k=0.01) and estimated using the given corpus. 

- If two or more candidates have the exact same perplexity in the input sentence, choose the one with the best edit distance. 
- If two or more candidates at the same perplexity also have the same edit distance, follow alphabetical order.

You should return a .json file containing a simple dictionary mapping the sentence ID (as an integer) to a tuple containing the best candidate replacement (as a string) and its perplexity under the specified language model (as a float). Not all candidate replacements might appear in the LM training corpus: don't map such candidate replacements to the UNK string though, or the perplexity estimate would not reflect the specific candidate; you can directly exclude candidate replacements which don't appear in the training corpus from the perplexity computation. 

5 points available, you lose 1 point for each wrong best candidate:frequency pair you retrieve (if the candidate is right but the perplexity doesn't match, you loose half a point).

In [53]:
# pick the best candidate according to perplexity under the given language model

lm = LM(LM_training_corpus, ngram_size=3, k=0.01)
lm.update_counts()

In [54]:
def replace_words(sentence, old_word, new_word):
    tokens = nltk.tokenize.word_tokenize(sentence)
    index = tokens.index(old_word)
    tokens[index] = new_word
    return tokens

In [60]:
best_candidates_perplexity = {}

for sen_id, neighbors in closest_neighbors.items():
    sentence = sentences_with_typos_df.loc[sentences_with_typos_df["ID"] == int(sen_id), "sentence"].iloc[0]
    mistyped_word = mistyped_words[sen_id]

    best_candidate = None
    best_perplexity = float("inf")
    
    for word, edit_distance in neighbors:
        if word in lm.vocab:
            replaced_sentence = replace_words(sentence, mistyped_word, word)
            perplexity = lm.perplexity(replaced_sentence)

            if perplexity < best_perplexity:
                best_perplexity = perplexity
                best_candidate = (word, edit_distance)
            elif perplexity == best_perplexity:
                if (edit_distance < best_candidate[1]) or (edit_distance == best_candidate[1] and word < best_candidate[0]):
                    best_candidate = (word, edit_distance)

    best_candidates_perplexity[sen_id] = (best_candidate[0], best_perplexity)


In [61]:
print(best_candidates_perplexity)

{1: ('know', 1002.7288999720108), 2: ('researchers', 14475.415924638553), 3: ('groin', 1858.1347148669051), 4: ('quote', 16820.32265338231), 5: ('waves', 1220.1171486647056), 6: ('wind', 1857.4043179891894), 7: ('jurors', 9336.433652424052), 8: ('survival', 18918.657040618258), 9: ('Squire', 2190.000743626073), 10: ('accommodate', 1897.8469004552046), 11: ('date', 2274.1923660774823), 12: ('colleague', 888.999722318408), 13: ('Fayed', 6625.174599990876), 14: ('care', 11882.810558938134), 15: ('iambic', 31291.168009478923)}


In [62]:
with open("task5_GhaithAlSeirawan_solution.json", "w") as f:
    json.dump(best_candidates_perplexity, f)
    

## Task 6

Compare the candidate replacements found when considering frequency and when considering perplexity. Which are better? Do they match what you consider to be the right replacement? What extra resources/information would you use to pick better candidate replacements? 3 points available, awarded based on how sensible the answer is.

### Note: the answer after one cell

In [75]:
# Display the found words for conveniance 

from IPython.display import display

candidate_replacements = {
    "ID": [],
    "Frequency": [],
    "Perplexity": [],
    "Grammerly.com": ["know", "researchers", "groan", "quilt", "waves", "winter", "minors", "surgically", "acquire", "acconmmodate", "dates", "colleague", "laid", "care", "iambic"]
}

for i in best_candidate_replacements:
    candidate_replacements["ID"].append(int(i))
    candidate_replacements["Frequency"].append(best_candidate_replacements[i][0])
    candidate_replacements["Perplexity"].append(best_candidates_perplexity[i][0])
    
candidate_replacements_df = pd.DataFrame(candidate_replacements)
candidate_replacements_df.set_index("ID", inplace = True)

display(candidate_replacements_df)

,Frequency,Perplexity,Grammerly.com
ID,,,
1,know,know,know
2,researchers,researchers,researchers
3,grown,groin,groan
4,quote,quote,quilt
5,waves,waves,waves
6,with,wind,winter
7,minor,jurors,minors
8,strictly,survival,surgically
9,Squire,Squire,acquire


### Answer

Comparing the candidate replacements found, I observed that some replacements are better than others. I manually checked the sentences for typos using Grammarly.com to compare both replacements found using frequency and perplexity. Only six words of the found replacements matched between frequency and perplexity. Perplexity replacements seem better in some cases, e.g., sentence 11: "data" instead of "days" or sentence 15: "iambic" instead of "gambit." 

Compared to Grammarly.com, both methods have limitations and need to be improved in finding the correct replacement. e.g., sentence 4: Grammarly suggested "quilt" but the frequency and perplexity "quote" or, sentence 7: Grammarly suggested "minors" but freq and perplexity "minor," "jurors" respectively.

A better replacement candidate might result from combining frequency and perplexity. This can be done by constructing a score measure defined as the product of given weights for the computed frequency and perplexity and multiplying by them. Considering both frequency and perplexity when choosing the best candidate replacements may lead to better results.

Given the volume of their training data, more advanced language models with BERT or GPT-based may also be able to provide better contextualized suggestions. Another method, like the Word@vec algorithm, that predicts the correct word using semantic similarity measures could also provide more accurate replacements.


#### The end